In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models

# Define the transforms for data preprocessing
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Set the path to your dataset
data_dir = '/content/drive/MyDrive/archive'

# Load the dataset and apply the transforms
train_dataset = datasets.ImageFolder( '/content/drive/MyDrive/archive/training set', transform=data_transforms)
val_dataset = datasets.ImageFolder( '/content/drive/MyDrive/archive/testing set', transform=data_transforms)

# Set the batch size for training and validation
batch_size = 32

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the number of classes
num_classes = len(train_dataset.classes)

# Create an instance of ResNet-50
model = models.resnet50(pretrained=True)

# Modify the fully connected layer for the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define the number of epochs for training
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Compute training statistics
        _, predicted = torch.max(outputs, 1)
        running_loss += loss.item()
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    # Validation phase
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Compute validation statistics
            _, predicted = torch.max(outputs, 1)
            running_loss += loss.item()
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    val_loss = running_loss / len(val_loader)
    val_accuracy = correct_predictions / total_predictions

    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} - "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 85.2MB/s]


Epoch [1/10] - Train Loss: 1.1426, Train Accuracy: 0.3129 - Val Loss: 1.2829, Val Accuracy: 0.3571
Epoch [2/10] - Train Loss: 0.9575, Train Accuracy: 0.6327 - Val Loss: 1.5124, Val Accuracy: 0.6190
Epoch [3/10] - Train Loss: 0.6803, Train Accuracy: 0.7891 - Val Loss: 1.0155, Val Accuracy: 0.6190
Epoch [4/10] - Train Loss: 0.4830, Train Accuracy: 0.9184 - Val Loss: 0.8359, Val Accuracy: 0.6667
Epoch [5/10] - Train Loss: 0.3230, Train Accuracy: 0.9524 - Val Loss: 1.0776, Val Accuracy: 0.6905
Epoch [6/10] - Train Loss: 0.1958, Train Accuracy: 0.9728 - Val Loss: 1.2327, Val Accuracy: 0.6429
Epoch [7/10] - Train Loss: 0.1629, Train Accuracy: 0.9728 - Val Loss: 1.1184, Val Accuracy: 0.6667
Epoch [8/10] - Train Loss: 0.0947, Train Accuracy: 0.9932 - Val Loss: 0.9344, Val Accuracy: 0.7381
Epoch [9/10] - Train Loss: 0.0677, Train Accuracy: 1.0000 - Val Loss: 0.8721, Val Accuracy: 0.7381
Epoch [10/10] - Train Loss: 0.0371, Train Accuracy: 1.0000 - Val Loss: 0.8627, Val Accuracy: 0.7381


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

image_path = '/content/drive/MyDrive/archive/testing set/Brown spot/DSC_0337.JPG'  # Change this to the path of your input image
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)

# Move the input tensor to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_batch = input_batch.to(device)

# Perform the prediction
with torch.no_grad():
    output = model(input_batch)

# Convert the output probabilities to predicted class labels
_, predicted_idx = torch.max(output, 1)
predicted_label = predicted_idx.item()

# Print the predicted label
print("Predicted Label:", predicted_label)

Predicted Label: 1
